In [1]:
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import grid_search_forecaster
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
holiday_events_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/holidays_events.csv')
stores_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/stores.csv')
transactions_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/transactions.csv')
train_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/train.csv')
test_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/test.csv')
oil_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/oil.csv')

In [3]:
holiday_events_df = holiday_events_df.sort_values(by='transferred', ascending=False).drop_duplicates(subset='date', keep='first')
holiday_events_df[holiday_events_df['transferred'] == True].head()
holiday_events_df.head()

,date,type,locale,locale_name,description,transferred
135,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
268,2016-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True
297,2017-01-01,Holiday,National,Ecuador,Primer dia del ano,True
312,2017-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
324,2017-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True


In [4]:
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [5]:
transactions_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [6]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [7]:
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].bfill()
oil_df.head()

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [33]:
full_train_df = train_df.merge(stores_df, how='left', on='store_nbr').merge(transactions_df, how='left', on=['date', 'store_nbr']).merge(oil_df, how='left', on='date').merge(holiday_events_df, how='left', on='date')
full_train_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False


In [34]:
full_test_df = test_df.merge(stores_df, how='left', on='store_nbr').merge(transactions_df, how='left', on=['date', 'store_nbr']).merge(oil_df, how='left', on='date').merge(holiday_events_df, how='left', on='date')
full_test_df.head()

,id,date,store_nbr,family,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y,locale,locale_name,description,transferred
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,46.8,NaN,NaN,NaN,NaN,NaN
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,46.8,NaN,NaN,NaN,NaN,NaN
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,46.8,NaN,NaN,NaN,NaN,NaN
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,46.8,NaN,NaN,NaN,NaN,NaN
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,NaN,46.8,NaN,NaN,NaN,NaN,NaN


In [35]:
full_train_df.dtypes

id                int64
date             object
store_nbr         int64
family           object
sales           float64
onpromotion       int64
city             object
state            object
type_x           object
cluster           int64
transactions    float64
dcoilwtico      float64
type_y           object
locale           object
locale_name      object
description      object
transferred      object
dtype: object

In [36]:
sample_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/sample_submission.csv')
sample_df.count()

id       28512
sales    28512
dtype: int64

In [37]:
test_df = pd.read_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/test.csv')
test_df.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [38]:
test_df.tail()

,id,date,store_nbr,family,onpromotion
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9
28511,3029399,2017-08-31,9,SEAFOOD,0


In [39]:
full_train_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False


In [40]:
# Re-format the date column and set it as index
full_train_df['date'] = pd.to_datetime(full_train_df['date'], format = '%Y-%m-%d')
# Re-format the date column and set it as index
full_test_df['date'] = pd.to_datetime(full_test_df['date'], format = '%Y-%m-%d')

In [41]:
full_train_df['day_of_week'] = full_train_df['date'].dt.dayofweek
full_test_df['day_of_week'] = full_test_df['date'].dt.dayofweek

In [42]:
full_train_df['month'] = full_train_df['date'].dt.month
full_test_df['month'] = full_test_df['date'].dt.month

In [43]:
# Reset the index to access 'date' column
full_train_df.reset_index(inplace=True)
full_test_df.reset_index(inplace=True)

# Set the date as index
full_train_df.set_index('date', inplace=True)
full_test_df.set_index('date', inplace=True)

In [44]:
# Set the dataset frequency to be (D)aily data
# full_train_df = full_train_df.asfreq('D', method = 'bfill') 
# full_test_df = full_test_df.asfreq('D', method = 'bfill') 
# Fill missing value with the latest available data

full_train_df.head()

,index,id,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y,locale,locale_name,description,transferred,day_of_week,month
date,,,,,,,,,,,,,,,,,,,
2013-01-01,0,0,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False,1,1
2013-01-01,1,1,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False,1,1
2013-01-01,2,2,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False,1,1
2013-01-01,3,3,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False,1,1
2013-01-01,4,4,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,93.14,Holiday,National,Ecuador,Primer dia del ano,False,1,1


In [45]:
full_test_df['type_y'].head()

date
2017-08-16    NaN
2017-08-16    NaN
2017-08-16    NaN
2017-08-16    NaN
2017-08-16    NaN
Name: type_y, dtype: object

In [46]:
# Encoding the type of products
family_label_encoder = LabelEncoder()
full_train_df['family_encoded'] = family_label_encoder.fit_transform(full_train_df['family'])
full_test_df['family_encoded'] = family_label_encoder.transform(full_test_df['family'])

#Encoding the citys and states
city_label_encoder = LabelEncoder()
full_train_df['city_encoded'] = city_label_encoder.fit_transform(full_train_df['city'])
full_test_df['city_encoded'] = city_label_encoder.transform(full_test_df['city'])

state_label_encoder = LabelEncoder()
full_train_df['state_encoded'] = state_label_encoder.fit_transform(full_train_df['state'])
full_test_df['state_encoded'] = state_label_encoder.transform(full_test_df['state'])

#Encoding the type of stores
type_store_label_encoder = LabelEncoder()
full_train_df['type_store_encoded'] = state_label_encoder.fit_transform(full_train_df['type_x'])
full_test_df['type_store_encoded'] = state_label_encoder.transform(full_test_df['type_x'])

#Transactions fillna
full_train_df['transactions'] = full_train_df['transactions'].fillna(0)
full_test_df['transactions'] = full_test_df['transactions'].fillna(0)
#Transactions Scale
transactions_scaler = MinMaxScaler()
full_train_df['transactions_scaled'] = transactions_scaler.fit_transform(full_train_df[['transactions']])
full_test_df['transactions_scaled'] = transactions_scaler.transform(full_test_df[['transactions']])

#Encoding the type of holidays
type_holiday_label_encoder = LabelEncoder()
full_train_df['type_y'].fillna('no_holiday', inplace=True)
full_test_df['type_y'].fillna('no_holiday', inplace=True)
combined_type_y = pd.concat([full_train_df['type_y'], full_test_df['type_y']])
type_holiday_label_encoder.fit(combined_type_y)
full_train_df['type_holiday_encoded'] = type_holiday_label_encoder.transform(full_train_df['type_y'])
full_test_df['type_holiday_encoded'] = type_holiday_label_encoder.transform(full_test_df['type_y'])

#Encoding the type of holidays
locale_label_encoder = LabelEncoder()
full_train_df['locale'].fillna('no_holiday', inplace=True)
full_test_df['locale'].fillna('no_holiday', inplace=True)
combined_type_y = pd.concat([full_train_df['locale'], full_test_df['locale']])
locale_label_encoder.fit(combined_type_y)
full_train_df['locale_encoded'] = locale_label_encoder.transform(full_train_df['locale'])
full_test_df['locale_encoded'] = locale_label_encoder.transform(full_test_df['locale'])


#Encoding the type of holidays
transferred_label_encoder = LabelEncoder()
full_train_df['transferred'].fillna('no_holiday', inplace=True)
full_test_df['transferred'].fillna('no_holiday', inplace=True)
full_train_df['transferred'].replace({True: 'transferred', False: 'not_transferred'}, inplace=True)
full_test_df['transferred'].replace({True: 'transferred', False: 'not_transferred'}, inplace=True)
combined_type_y = pd.concat([full_train_df['transferred'], full_test_df['transferred']])
transferred_label_encoder.fit(combined_type_y)
full_train_df['transferred_encoded'] = transferred_label_encoder.transform(full_train_df['transferred'])
full_test_df['transferred_encoded'] = transferred_label_encoder.transform(full_test_df['transferred'])

oil_scaler = MinMaxScaler()
full_train_df['dcoilwtico'] = full_train_df['dcoilwtico'].bfill()
full_test_df['dcoilwtico'] = full_test_df['dcoilwtico'].bfill()
full_train_df['dcoilwtico_scaled'] = oil_scaler.fit_transform(full_train_df[['dcoilwtico']])
full_test_df['dcoilwtico_scaled'] = oil_scaler.transform(full_test_df[['dcoilwtico']])

sales_scaler = MinMaxScaler()
full_train_df['sales_scaled'] = sales_scaler.fit_transform(full_train_df[['sales']])
# full_test_df['sales_scaled'] = sales_scaler.transform(full_test_df[['sales']])



/tmp/ipykernel_76554/3519454474.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_train_df['type_y'].fillna('no_holiday', inplace=True)
/tmp/ipykernel_76554/3519454474.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [47]:
full_train_df.head()


,index,id,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,family_encoded,city_encoded,state_encoded,type_store_encoded,transactions_scaled,type_holiday_encoded,locale_encoded,transferred_encoded,dcoilwtico_scaled,sales_scaled
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0,0,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,0,18,12,3,0.0,3,1,1,0.792965,0.0
2013-01-01,1,1,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,...,1,18,12,3,0.0,3,1,1,0.792965,0.0
2013-01-01,2,2,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,...,2,18,12,3,0.0,3,1,1,0.792965,0.0
2013-01-01,3,3,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,...,3,18,12,3,0.0,3,1,1,0.792965,0.0
2013-01-01,4,4,1,BOOKS,0.0,0,Quito,Pichincha,D,13,...,4,18,12,3,0.0,3,1,1,0.792965,0.0


In [48]:
full_test_df.head()

,index,id,store_nbr,family,onpromotion,city,state,type_x,cluster,transactions,...,month,family_encoded,city_encoded,state_encoded,type_store_encoded,transactions_scaled,type_holiday_encoded,locale_encoded,transferred_encoded,dcoilwtico_scaled
date,,,,,,,,,,,,,,,,,,,,,
2017-08-16,0,3000888,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,0.0,...,8,0,18,12,3,0.0,6,3,0,0.244108
2017-08-16,1,3000889,1,BABY CARE,0,Quito,Pichincha,D,13,0.0,...,8,1,18,12,3,0.0,6,3,0,0.244108
2017-08-16,2,3000890,1,BEAUTY,2,Quito,Pichincha,D,13,0.0,...,8,2,18,12,3,0.0,6,3,0,0.244108
2017-08-16,3,3000891,1,BEVERAGES,20,Quito,Pichincha,D,13,0.0,...,8,3,18,12,3,0.0,6,3,0,0.244108
2017-08-16,4,3000892,1,BOOKS,0,Quito,Pichincha,D,13,0.0,...,8,4,18,12,3,0.0,6,3,0,0.244108


In [311]:
train_data=full_train_df[['store_nbr', 'onpromotion', 'cluster', 'day_of_week', 'month', 'family_encoded', 'city_encoded', 'state_encoded', 'type_store_encoded', 'type_holiday_encoded', 'locale_encoded', 'transferred_encoded', 'dcoilwtico', 'sales']]
test_data=full_test_df[['store_nbr', 'onpromotion', 'cluster', 'day_of_week', 'month', 'family_encoded', 'city_encoded', 'state_encoded', 'type_store_encoded', 'type_holiday_encoded', 'locale_encoded', 'transferred_encoded', 'dcoilwtico']]

# Ensure the date ranges are correctly defined
train_data_start=pd.to_datetime('2013-01-01')
train_data_end=pd.to_datetime('2017-08-15')

test_data_start=pd.to_datetime('2017-08-16')
test_data_end=pd.to_datetime('2017-08-31')

train_data.head()



,store_nbr,onpromotion,cluster,day_of_week,month,family_encoded,city_encoded,state_encoded,type_store_encoded,type_holiday_encoded,locale_encoded,transferred_encoded,dcoilwtico,sales
date,,,,,,,,,,,,,,
2013-01-01,1,0,13,1,1,0,18,12,3,3,1,1,93.14,0.0
2013-01-01,1,0,13,1,1,1,18,12,3,3,1,1,93.14,0.0
2013-01-01,1,0,13,1,1,2,18,12,3,3,1,1,93.14,0.0
2013-01-01,1,0,13,1,1,3,18,12,3,3,1,1,93.14,0.0
2013-01-01,1,0,13,1,1,4,18,12,3,3,1,1,93.14,0.0


In [312]:
# Group the train_data by 'store_nbr' and 'family'
grouped_train = train_data.groupby(['store_nbr', 'family_encoded'])
# Group the train_data by 'store_nbr' and 'family'
grouped_test = test_data.groupby(['store_nbr', 'family_encoded'])

# Create an array of DataFrames
dataframes_array_train = [group for _, group in grouped_train]

# Create an array of DataFrames
dataframes_array_test = [group for _, group in grouped_test]




In [271]:
dataframes_array_train[0].head()

,store_nbr,onpromotion,cluster,day_of_week,month,family_encoded,city_encoded,state_encoded,type_store_encoded,type_holiday_encoded,locale_encoded,transferred_encoded,dcoilwtico_scaled,sales_scaled
date,,,,,,,,,,,,,,
2013-01-01,1,0,13,1,1,0,18,12,3,3,1,1,0.792965,0.000000
2013-01-02,1,0,13,2,1,0,18,12,3,6,3,0,0.792965,0.000016
2013-01-03,1,0,13,3,1,0,18,12,3,6,3,0,0.790951,0.000024
2013-01-04,1,0,13,4,1,0,18,12,3,6,3,0,0.792728,0.000024
2013-01-05,1,0,13,5,1,0,18,12,3,5,1,1,0.793675,0.000040


In [313]:
# Add a row with the day 2012-12-25 to each DataFrame in dataframes_array_train
for idx, df in enumerate(dataframes_array_train):
    for i in range(2013, 2017):
        new_row = pd.DataFrame({
            'date': [pd.to_datetime(f'{i}-12-25')],
            'store_nbr': [df['store_nbr'].iloc[0]],
            'onpromotion': [0],
            'cluster': [df['cluster'].iloc[0]],
            'day_of_week': [df.loc[f'{i}-12-24', 'day_of_week'] + 1 if df.loc[f'{i}-12-24', 'day_of_week'] + 1 != 8 else 1],  # Assuming 1 for Monday, adjust as needed
            'month': [12],
            'family_encoded': [df['family_encoded'].iloc[0]],
            'city_encoded': [df['city_encoded'].iloc[0]],
            'state_encoded': [df['state_encoded'].iloc[0]],
            'type_store_encoded': [df['type_store_encoded'].iloc[0]],
            'type_holiday_encoded': [1],  # Assuming 1 for holiday, adjust as needed
            'locale_encoded': [1],  # Assuming 1 for locale, adjust as needed
            'transferred_encoded': [0],  # Assuming 0 for not transferred, adjust as needed
            'dcoilwtico': [df.loc[f'{i}-12-24', 'dcoilwtico']],  # Assuming a default value, adjust as needed
            'sales': [0]  # Assuming no sales, adjust as needed
        })
        new_row.set_index('date', inplace=True)
        df = pd.concat([df, new_row])
        df.sort_index(inplace=True)
        #df.reset_index(inplace=True)
        #df.set_index('date', inplace=True)
    dataframes_array_train[idx] = df



In [314]:
dataframes_array_train[0].head()

,store_nbr,onpromotion,cluster,day_of_week,month,family_encoded,city_encoded,state_encoded,type_store_encoded,type_holiday_encoded,locale_encoded,transferred_encoded,dcoilwtico,sales
date,,,,,,,,,,,,,,
2013-01-01,1,0,13,1.0,1,0,18,12,3,3,1,1,93.14,0.0
2013-01-02,1,0,13,2.0,1,0,18,12,3,6,3,0,93.14,2.0
2013-01-03,1,0,13,3.0,1,0,18,12,3,6,3,0,92.97,3.0
2013-01-04,1,0,13,4.0,1,0,18,12,3,6,3,0,93.12,3.0
2013-01-05,1,0,13,5.0,1,0,18,12,3,5,1,1,93.20,5.0


In [315]:
# Generate a complete date range
complete_date_range = pd.date_range(start=train_data_start, end=train_data_end)

# Find missing dates
missing_dates = complete_date_range.difference(dataframes_array_train[0].loc[train_data_start:train_data_end].index)

# Display missing dates
print(missing_dates)

DatetimeIndex([], dtype='datetime64[ns]', freq='D')


In [316]:
# Keep only unique rows in each DataFrame in dataframes_array_train
dataframes_array_train = [df.drop_duplicates(keep='first') for df in dataframes_array_train]

# Fill missing dates in each DataFrame in dataframes_array_train
for df in dataframes_array_train:
#    complete_date_range = pd.date_range(start=df.index.min(), end=df.index.max())
#    df = df.reindex(complete_date_range, method='ffill')
#    df.sort_index(inplace=True)
    df.asfreq('D', method='bfill')

# Display the first few DataFrames in the array
for df in dataframes_array_train[:3]:
    print(df.head())

for df in dataframes_array_test:
#    complete_date_range = pd.date_range(start=df.index.min(), end=df.index.max())
#    df = df.reindex(complete_date_range, method='ffill')
#    df.sort_index(inplace=True)
    df.asfreq('D', method='bfill')

# Display the first few DataFrames in the array
for df in dataframes_array_train[:3]:
    print(df.head())

# Create an array of DataFrames
dataframes_array_test = [group for _, group in grouped_test]
for df in dataframes_array_test[:3]:
    print(df.head())
    



            store_nbr  onpromotion  cluster  day_of_week  month  \
date                                                              
2013-01-01          1            0       13          1.0      1   
2013-01-02          1            0       13          2.0      1   
2013-01-03          1            0       13          3.0      1   
2013-01-04          1            0       13          4.0      1   
2013-01-05          1            0       13          5.0      1   

            family_encoded  city_encoded  state_encoded  type_store_encoded  \
date                                                                          
2013-01-01               0            18             12                   3   
2013-01-02               0            18             12                   3   
2013-01-03               0            18             12                   3   
2013-01-04               0            18             12                   3   
2013-01-05               0            18             12 

In [317]:
# Check for duplicate dates in each DataFrame in dataframes_array_train
# for idx, df in enumerate(dataframes_array_train):
#    duplicates = df.index.duplicated(keep=False)
#    if duplicates.any():
#        print(f"DataFrame at index {idx} has duplicate dates:")
#        print(df[duplicates])

# Check for duplicate dates in each DataFrame in dataframes_array_train
# for idx, df in enumerate(dataframes_array_train):
#    duplicates = df.index.duplicated(keep=False)
#    if duplicates.any():
#        print(f"DataFrame at index {idx} has duplicate dates:")
#        print(df[duplicates])

dataframes_array_train[0].loc['2013-05-12']

store_nbr                1.00
onpromotion              0.00
cluster                 13.00
day_of_week              6.00
month                    5.00
family_encoded           0.00
city_encoded            18.00
state_encoded           12.00
type_store_encoded       3.00
type_holiday_encoded     2.00
locale_encoded           1.00
transferred_encoded      1.00
dcoilwtico              94.76
sales                    1.00
Name: 2013-05-12 00:00:00, dtype: float64

In [318]:


target = {}
for idx, df in enumerate(dataframes_array_train):
    target[idx] = df.loc[train_data_start:train_data_end, df.columns == 'sales']
    
target[0].head()


,sales
date,
2013-01-01,0.0
2013-01-02,2.0
2013-01-03,3.0
2013-01-04,3.0
2013-01-05,5.0


In [319]:
# Complete target with 0 between test_data_start and test_data_end
for idx, df in target.items():
    complete_date_range = pd.date_range(start=test_data_start, end=test_data_end)
    missing_dates = complete_date_range.difference(df.index)
    for date in missing_dates:
        df.loc[date] = 0
    df.sort_index(inplace=True)
    df.asfreq('D', method='bfill')

    


In [320]:
target[0].tail()

,sales
date,
2017-08-27,0.0
2017-08-28,0.0
2017-08-29,0.0
2017-08-30,0.0
2017-08-31,0.0


In [321]:
for df in dataframes_array_train:
    df.drop(columns=['sales'], inplace=True)


In [66]:
len(dataframes_array_train)

1782

In [67]:
len(dataframes_array_test)

1782

In [322]:
for c in range(0, len(dataframes_array_train)):
    dataframes_array_train[c] = pd.concat([dataframes_array_train[c], dataframes_array_test[c]])
    

In [324]:

# Generate a complete date range
complete_date_range = pd.date_range(start=train_data_start, end=train_data_end)

# Find missing dates
missing_dates = complete_date_range.difference(target[0].loc[train_data_start:train_data_end].index)

# Display missing dates
missing_dates


DatetimeIndex([], dtype='datetime64[ns]', freq='D')

In [325]:
print(target[0].loc[train_data_start:test_data_end, 'sales'].head())
print(target[0].loc[train_data_start:test_data_end, 'sales'].tail())

date
2013-01-01    0.0
2013-01-02    2.0
2013-01-03    3.0
2013-01-04    3.0
2013-01-05    5.0
Name: sales, dtype: float64
date
2017-08-27    0.0
2017-08-28    0.0
2017-08-29    0.0
2017-08-30    0.0
2017-08-31    0.0
Name: sales, dtype: float64


In [247]:

for df in target.values():
    df.index = pd.to_datetime(df.index)
    df.asfreq('D', method='bfill')
    
target[0].head()


,sales_scaled
date,
2013-01-01,0.000000
2013-01-02,0.000016
2013-01-03,0.000024
2013-01-04,0.000024
2013-01-05,0.000040


In [329]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor
# Define the forecaster with multiple series
forecaster = ForecasterAutoreg(
    regressor = DecisionTreeRegressor(random_state = 123,
                                      ),
    lags = 30,
)



In [345]:
# Parameter Grid for Regressor
param_grid = {
    # max_depth: The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.
    'max_depth' : [None, 1, 3, 5],
    # min_samples_split: The minimum number of samples required to split an internal node.
    'min_samples_split' : [2, 3, 4, 5, 6, 10],
    # ccp_alpha: Complexity parameter used for Minimal Cost-Complexity Pruning. The subtree with the largest cost complexity that is smaller than ccp_alpha will be chosen.
    'ccp_alpha' : [0.0, 0.001, 0.01]
}

# lags_grid: The number of previous observations to consider
lags_grid = [30, 50, 100, 150, [1,2,3,4,5,7,9,13,15,20,50], 10, 20]


exog = dataframes_array_train[0].loc[train_data_start:train_data_end, ['store_nbr', 'onpromotion', 'cluster', 'day_of_week', 'month', 'family_encoded', 'city_encoded', 'state_encoded', 'type_store_encoded', 'type_holiday_encoded', 'locale_encoded', 'transferred_encoded', 'dcoilwtico']]
exog = exog.asfreq('D')
target_test=target[0].loc[train_data_start:train_data_end, 'sales']
target_test=target_test.asfreq('D')


# Grid Search with Refit and Increasing Train Size
grid_forecaster = grid_search_forecaster(
    # forecaster - our model
    forecaster = forecaster,
    # y - target variable
    y = target_test,
    # exog - exogenous variables
    exog = exog,
    # param_grid - dictionary with parameters to test
    param_grid = param_grid,
    # lag_grid - list with lags to test
    lags_grid = lags_grid,
    # steps - number of steps to forecast
    steps = len(dataframes_array_train[0].loc[test_data_start:test_data_end]),
    # refit - refit the best model with all data
    refit = True,
    # metric - metric to optimize
    metric = 'mean_squared_error',
    # initial_train_size - initial size of the train data
    initial_train_size = len(dataframes_array_train[0].loc[train_data_start:train_data_end])-1,
    # fixed_train_size - if True, the training size is fixed
    fixed_train_size = False,
    # return_best - return the best model
    return_best = True,
    # verbose - show all messages
    verbose = False
)

# Backtest RMSE
np.sqrt(grid_forecaster.iloc[0]['mean_squared_error'])

Number of models compared: 504.


lags grid: 100%|████████████████████████████████| 7/7 [00:31<00:00,  4.44s/it]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10] 
  Parameters: {'ccp_alpha': 0.0, 'max_depth': None, 'min_samples_split': 4}
  Backtesting metric: 0.0




/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(


0.0

In [353]:

target_test=target[1].loc[train_data_start:train_data_end, 'sales']
target_test=target_test.asfreq('D')
target_test.head()


date
2013-01-01    0.0
2013-01-02    0.0
2013-01-03    0.0
2013-01-04    0.0
2013-01-05    0.0
Freq: D, Name: sales, dtype: float64

In [358]:


final_df = pd.DataFrame()

for i in range(0, len(dataframes_array_train)):
    exog = dataframes_array_train[i].loc[train_data_start:train_data_end, ['store_nbr', 'onpromotion', 'cluster', 'day_of_week', 'month', 'family_encoded', 'city_encoded', 'state_encoded', 'type_store_encoded', 'type_holiday_encoded', 'locale_encoded', 'transferred_encoded', 'dcoilwtico']]
    exog = exog.asfreq('D')
    target_test=target[i].loc[train_data_start:train_data_end, 'sales']
    target_test=target_test.asfreq('D').fillna(method='ffill')  # Fill missing values
    forecaster.fit(y=target_test, exog=exog)
    # prediction
    exog = dataframes_array_train[i].loc[test_data_start:test_data_end, ['store_nbr', 'onpromotion', 'cluster', 'day_of_week', 'month', 'family_encoded', 'city_encoded', 'state_encoded', 'type_store_encoded', 'type_holiday_encoded', 'locale_encoded', 'transferred_encoded', 'dcoilwtico']]
    exog.index = pd.to_datetime(exog.index)
    exog = exog.asfreq('D')
    predicted_values = forecaster.predict(steps=len(dataframes_array_train[i].loc[test_data_start:test_data_end]), exog=exog)
    predicted_values = pd.DataFrame({'store_nbr': [dataframes_array_train[i]['store_nbr'].iloc[0]], 'family_encoded': [dataframes_array_train[i]['family_encoded'].iloc[0]], 'sales': predicted_values}, index=exog.index)
    final_df = pd.concat([final_df, predicted_values])
    
# Fit the model using train data with multiple series
#exog = dataframes_array_train[0].loc[train_data_start:train_data_end, ['store_nbr', 'onpromotion', 'cluster', 'day_of_week', 'month', 'family_encoded', 'city_encoded', 'state_encoded', 'type_store_encoded', 'type_holiday_encoded', 'locale_encoded', 'transferred_encoded', 'dcoilwtico']]
#exog = exog.asfreq('D')
#target_test=target[0].loc[train_data_start:train_data_end, 'sales']
#target_test=target_test.asfreq('D')
#forecaster.fit(y=target_test, exog=exog)




/tmp/ipykernel_76554/2706406145.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  target_test=target_test.asfreq('D').fillna(method='ffill')  # Fill missing values
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_discretization.py:307: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn(
/tmp/ipykernel_76554/2706406145.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  target_test=target_test.asfreq('D').fillna(method='ffill')  # Fill missing values
/usr/local/lib/python3.10/dist-packages/skforecast/utils/utils.py:450: MissingValuesWarning: `exog` has missing values. Most machine learning models do not allow missing values. Fitting the forecaster may fail. 
 You can suppress this warning using: warnings.si

In [359]:
full_test_df.head()

,index,id,store_nbr,family,onpromotion,city,state,type_x,cluster,transactions,...,month,family_encoded,city_encoded,state_encoded,type_store_encoded,transactions_scaled,type_holiday_encoded,locale_encoded,transferred_encoded,dcoilwtico_scaled
date,,,,,,,,,,,,,,,,,,,,,
2017-08-16,0,3000888,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,0.0,...,8,0,18,12,3,0.0,6,3,0,0.244108
2017-08-16,1,3000889,1,BABY CARE,0,Quito,Pichincha,D,13,0.0,...,8,1,18,12,3,0.0,6,3,0,0.244108
2017-08-16,2,3000890,1,BEAUTY,2,Quito,Pichincha,D,13,0.0,...,8,2,18,12,3,0.0,6,3,0,0.244108
2017-08-16,3,3000891,1,BEVERAGES,20,Quito,Pichincha,D,13,0.0,...,8,3,18,12,3,0.0,6,3,0,0.244108
2017-08-16,4,3000892,1,BOOKS,0,Quito,Pichincha,D,13,0.0,...,8,4,18,12,3,0.0,6,3,0,0.244108


In [360]:
# Ensure the date column is in the index for both dataframes
final_df.reset_index(inplace=True)
full_test_df.reset_index(inplace=True)

# Merge the dataframes on 'family_encoded', 'store_nbr', and 'date'
merged_df = pd.merge(final_df, full_test_df, on=['family_encoded', 'store_nbr', 'date'], how='inner')

# Display the merged dataframe
merged_df.head()

,date,store_nbr,family_encoded,sales,index,id,family,onpromotion,city,state,...,day_of_week,month,city_encoded,state_encoded,type_store_encoded,transactions_scaled,type_holiday_encoded,locale_encoded,transferred_encoded,dcoilwtico_scaled
0,2017-08-16,1,0,19.0,0,3000888,AUTOMOTIVE,0,Quito,Pichincha,...,2,8,18,12,3,0.0,6,3,0,0.244108
1,2017-08-17,1,0,4.0,1782,3002670,AUTOMOTIVE,0,Quito,Pichincha,...,3,8,18,12,3,0.0,6,3,0,0.247305
2,2017-08-18,1,0,8.5,3564,3004452,AUTOMOTIVE,0,Quito,Pichincha,...,4,8,18,12,3,0.0,6,3,0,0.265309
3,2017-08-19,1,0,6.0,5346,3006234,AUTOMOTIVE,0,Quito,Pichincha,...,5,8,18,12,3,0.0,6,3,0,0.251096
4,2017-08-20,1,0,2.0,7128,3008016,AUTOMOTIVE,0,Quito,Pichincha,...,6,8,18,12,3,0.0,6,3,0,0.251096


In [362]:
submission_df=merged_df[['id', 'sales']]
submission_df.head()

,id,sales
0,3000888,19.0
1,3002670,4.0
2,3004452,8.5
3,3006234,6.0
4,3008016,2.0


In [363]:
!pwd

/mnt/c/Users/agust/Documents/GitHub/Data-Science/Data-Science


In [364]:
submission_df.to_csv('RNNs/Tensorflow-Keras/Predictions/Store Sales - Time Series Forecasting/submission_2.csv', index=False)